In [4]:
import time

In [5]:
DB = CremonaDatabase(); DB

Cremona's database of elliptic curves with conductor at most 399998

In [3]:
t0 = time.time()
p = 2
labels = []
for i in range(5000, 6000):
    N = p*i
    D = DB.allcurves(N)
    for e in D:
        labels += ['{}{}'.format(N,e)]
        
# need to impose bad additive reduction and j valuation >= 0
print len(labels)
new_labels = []
for label in labels:
    E = EllipticCurve(label)
    if valuation(E.j_invariant(),p) >= 0 and E.has_additive_reduction(p):
        # has_additive_reduction might just run Tate's alg. if so, save results
        new_labels += [label]
labels = new_labels
t1 = time.time()

print len(labels)
print("Time: {}".format(t1-t0))

NameError: name 'time' is not defined

In [5]:
# coarse kraus for over Q2
E = EllipticCurve('24a3')
disc_val = valuation(E.discriminant(), 2)
c4_val  = valuation(E.c4(), 2)
if E.kodaira_type(2) in [KodairaSymbol('IV'), KodairaSymbol('IV*')]:
    print '|Φ| = 3'
elif disc_val == 3*c4_val:
    print '|Φ| = 2'
elif disc_val < 3*c4_val < 12 + disc_val:
    print '4 divides (v(c_4)|Φ|)'
elif 3*c4_val >= 12 + disc_val:
    if disc_val % 3 == 0:
        print '|Φ| = 2'
    else:
        print '|Φ| = 6'
else:
    print "terminated without an answer"

4 divides (v(c_4)|Φ|)


In [6]:
# kraus decision tree for Q2 (just coding a table)
# to determine group order (hence group, in this case) just need:
# c invariants, their 2-prime parts, their valuations, their reductions mod 4
# discriminant, its 2-prime part, its valuation, and its reduction mod 4
def KrausTableQ2(E):
    (c4, c6) = E.c_invariants()
    D  = E.discriminant()

    c4u = c4.prime_to_S_part([2])
    c6u = c6.prime_to_S_part([2])
    Du = D.prime_to_S_part([2])

    vD = valuation(D, 2)
    vc4 = valuation(c4, 2)
    vc6 = valuation(c6, 2)

    c4ured = c4u % 4
    c6ured = c6u % 4
    Dured = Du % 4
    
    if vD < 4 or vD == 5 or vD > 18:
        print("Error: invalid discriminant valuation")

    if vD == 4:
        if vc4 == 4:
            if c4ured == 3:
                if c6ured == 1:
                    return 3
                else:
                    return 6
            else:
                return 24
        if vc4 == 5:
            return 24
        else:
            if c6ured == 1:
                return 3
            else:
                return 6

    elif vD == 6:
        if vc4 == 4:
            return 8 
        elif vc4 == 5:
            return 8
        else:
            return 2

    elif vD == 7:
        if vc4 == 4:
            return 24
        else:
            print "This shouldn't happen."
    
    elif vD == 8:
        if vc4 == 4:
            if Dured == 3:
                if c6ured == 1:
                    return 6
                else:
                    return 3
            else:
                return 24
        elif vc4 == 5:
            return 24
        elif vc4 == 6:
            return 24
        else:
            if c6ured == 1:
                return 3
            else:
                return 6
    
    elif vD == 9:
        if vc4 == 4:
            return 8
        elif vc4 == 5:
            if vc6 == 8:
                return 4
            if vc6 > 8:
                return 8
            else:
                print "This shouldn't happen."
        else:
            print "This shouldn't happen."
    
    elif vD == 10:
        if vc4 == 4:
            return 24
        elif vc4 == 5:
            print "This shouldn't happen."
        elif vc4 == 6:
            if c4ured == 3:
                return 6
            else:
                return 24
        elif vc4 == 7:
            return 24
        else:
            return 6
    elif vD == 11:
        if vc4 == 4:
            return 24
        else:
            print "This shouldn't happen."
    elif vD == 12:
        if vc4 == 4:
            return 2
        elif vc4 == 6:
            return 8
        elif vc4 == 7:
            return 8
        elif vc4 == 8:
            return 2
        else:
            return 2      

    elif vD == 13:
        if vc4 == 6:
            return 24
        else:
            print "This shouldn't happen."

    elif vD == 14:
        if vc4 == 6:
            if Dured == 3:
                return 6
            else:
                return 24
        elif vc4 == 7:
            return 24
        elif vc4 == 8:
            return 24
        elif vc4 > 8:
            return 6
        else:
            print "This shouldn't happen."

    elif vD == 15:
        if vc4 == 6:
            return 8
        else:
            if vc6 == 11:
                return 4
            else:
                return 8
            
    elif vD == 16:
        if vc4 == 6:
            return 24
        else:
            print "This shouldn't happen."

    elif vD == 17:
        if vc4 == 6:
            return 24
        else:
            print "This shouldn't happen."

    else:
        if vc4 == 6:
            return 2
        else:
            print "This shouldn't happen."    

In [7]:
# # for testing KrausTableQ2
# S = []
# for label in labels:
#     E = EllipticCurve(label)
#     vD = valuation(E.discriminant(),2)
#     S += [label]
# #     if vD == 12:
# #         S += [label]
# for label in S:
#     E = EllipticCurve(label)
#     KrausTableQ2(E)

In [8]:
########## first investivation ############

In [9]:
monodromy_types = [2, 3, 4, 6, 8, 24]

In [10]:
# generate a bunch of elliptic curves with Q8 monodromy and compute kodaira types
labels_grouped = []
for label in labels:
    E = EllipticCurve(label)
    labels_grouped.append((KrausTableQ2(E), label))

In [611]:
monodromy_dict = dict.fromkeys(monodromy_types, [])
for pair in labels_grouped:
    monodromy_dict[pair[0]] += [pair[1]]

In [614]:
# testing: find a curve with type IV
k_types_seen = []
for label in monodromy_dict[8]:
    E = EllipticCurve(label)
    k_type_E = str(E.kodaira_symbol(2))
    if k_type_E == "IV":
        print label
        break

20a2


In [597]:
E = EllipticCurve('20a2')
# this is supposed to have kodaira type IV but Q8 monodromy
# this is bad because Q8 does not admit C_3 as a quotient or a sub

E.local_data(2)
# right, so type is IV
# now I should check the monodromy extension by hand

(c4, c6) = E.c_invariants()
D  = E.discriminant()
c4u = c4.prime_to_S_part([2])
c6u = c6.prime_to_S_part([2])
Du = D.prime_to_S_part([2])
vD = valuation(D, 2)
vc4 = valuation(c4, 2)
vc6 = valuation(c6, 2)
c4ured = c4u % 4
c6ured = c6u % 4
Dured = Du % 4

valuation(E.j_invariant(), 2)
# valuation of j is 14 and red is bad additive so Kraus applies
vD
# valuation of discriminant is 4
vc4
# valuation of c_4 is 6, so I need to check c'_6 mod 4
c6ured
# valuation of c'_6 is 1, so monodromy group has size 3.
# so my coding sucks but the dream is alive!
# KrausTableQ2 gets this right, but it's grouped incorrectly

1

In [11]:
########### second attempt ############

In [12]:
# compute monodromy types of all labels
# group in a dictionary where keys are the monodromy types
# a monodromy type is a number 2, 3, 4, 6, 8, 24, the order of the monodromy group
# if 2, 3, 4, or 6, the group is cyclic
# if 8, the group is Q8
# if 24, the group is SL_2(F_3)

t0 = time.time()
m_types = [2, 3, 4, 6, 8, 24]
m_dict = dict.fromkeys(m_types, [])

# group labels by monodromy type
for label in labels:
    E = EllipticCurve(label)
    m_type = KrausTableQ2(E)
    m_dict[m_type] = m_dict[m_type] + [label]
    
t1 = time.time()
print("Time: {}".format(t1-t0))

Time: 9.82452583313


In [13]:
# for each curve,  compute the kodaira type
# output: a dictonary with keys the monodromy types and entries the kodaira types
#         another dictionary with the same keys and entries (kodaira type, label)
t0 = time.time()
grouped_k_types = dict.fromkeys(m_types, [])
m_k_egs = dict.fromkeys(m_types, [])
for m_type in m_types:
    for label in m_dict[m_type]:
        E = EllipticCurve(label)
        k_type = E.kodaira_type(p)
        if k_type not in grouped_k_types[m_type]:
            grouped_k_types[m_type] = grouped_k_types[m_type] + [k_type]
            m_k_egs[m_type] = m_k_egs[m_type] + [(k_type, label)]
t1 = time.time()
print("Time: {}".format(t1-t0))

Time: 12.9623780251


In [14]:
grouped_k_types

{2: [II*, I4*, II, I8*],
 3: [IV*, IV],
 4: [III, III*],
 6: [II, I0*, I4*, II*],
 8: [I3*, I0*, III, I2*, I5*, II, III*],
 24: [I1*, III*, II*, III, I0*, I2*, I3*, II, I7*, I6*, I4*]}

In [15]:
def k_type_to_cpt_group(symbol):
    if symbol in [KodairaSymbol('II'), KodairaSymbol('II*')]:
        return "0"
    elif symbol in [KodairaSymbol('III'), KodairaSymbol('III*')]:
        return "Z/2"
    elif symbol in [KodairaSymbol('IV'), KodairaSymbol('IV*')]:
        return "Z/3"
    elif str(symbol)[-1] == '*':
        if int(str(symbol)[1:-1]) % 2 == 0:
            return "Z/2 x Z/2"
        else:
            return "Z/4"
    else:
        print "Invalid reduction type"
    

In [16]:
for m_type in grouped_k_types.keys():
    cpt_gps = map(lambda t: k_type_to_cpt_group(t), grouped_k_types[m_type])
    cpt_gps = list(set(cpt_gps))
    print(m_type, cpt_gps)

(2, ['0', 'Z/2 x Z/2'])
(3, ['Z/3'])
(4, ['Z/2'])
(6, ['0', 'Z/2 x Z/2'])
(8, ['Z/2 x Z/2', 'Z/2', 'Z/4', '0'])
(24, ['0', 'Z/2', 'Z/4', 'Z/2 x Z/2'])


In [17]:
m_k_egs

{2: [(II*, '176b1'), (I4*, '240d8'), (II, '704k3'), (I8*, '960i8')],
 3: [(IV*, '20a1'), (IV, '20a2')],
 4: [(III, '256a1'), (III*, '256a2')],
 6: [(II, '80b4'), (I0*, '80b3'), (I4*, '320f2'), (II*, '704f1')],
 8: [(I3*, '32a1'),
  (I0*, '32a3'),
  (III, '32a2'),
  (I2*, '64a1'),
  (I5*, '64a3'),
  (II, '64a4'),
  (III*, '256b2')],
 24: [(I1*, '24a1'),
  (III*, '24a3'),
  (II*, '24a5'),
  (III, '24a4'),
  (I0*, '48a1'),
  (I2*, '48a3'),
  (I3*, '48a5'),
  (II, '48a4'),
  (I7*, '192d5'),
  (I6*, '192d4'),
  (I4*, '192c2')]}

In [761]:
# remarks:
# McCallum proves exponent of inertial monodromy kills component group
# exponents: 2, 3, 4, 6, 8, 24 |-> 2, 3, 4, 6, 4, 12
#
# for C_2, the possible component group are: 0, C_2, C_2 x C_2,
#        but we don't see C_2
# note: in this case, monodromy group smaller than component group
# note: I think Dokchitser paper shows this is the only situation where potentially ordinary
#       reduction is possible.
#
# for C_3, possibilities are 0 and C_3, 
#        but we don't see 0
#
# for C_4, possibilities are 0, C_2, C_4, C_2 x C_2,
#        but we ONLY see one of these, C_2
#
# for C_6, possibilities are 0, C_2, C_3, C_2 x C_2,
#        but we don't see C_2 or C_3
# note: C_2 x C_2 is not a subquotient of C_6
# note: Dokchitsers show this is potentially supersingular
#
# for Q8, the possiblities are 0, C_2, C_4, C_2 x C_2, and we see all of these
# 
# for SL_2(F_3), the possiblities are 0, C_2, C_4, C_2 x C_2, and wee see all of these

In [877]:
# check by hand that there is a curve with reduction I4star and monodromy group C6
# find an example
# deprecated: I log one example for each curve now
I4star_C6_labels = []
for label in m_dict[6]:
    E = EllipticCurve(label)
    if E.kodaira_symbol(p) == KodairaSymbol('I4*'):
        I4star_C6_labels += [label]
I4star_C6_labels[:5]

['320f2', '320f4', '320c2', '320c4', '576e2']

In [858]:
# choose one of thes
E = EllipticCurve('320f2'); E

Elliptic Curve defined by y^2 = x^3 + x^2 + 15*x - 17 over Rational Field

In [859]:
E.local_data(2) # check: types is I4*

(c4, c6) = E.c_invariants()
D  = E.discriminant()
c4u = c4.prime_to_S_part([2])
c6u = c6.prime_to_S_part([2])
Du = D.prime_to_S_part([2])
vD = valuation(D, 2)
vc4 = valuation(c4, 2)
vc6 = valuation(c6, 2)
c4ured = c4u % 4
c6ured = c6u % 4
Dured = Du % 4

valuation(E.j_invariant(), 2) 
# check: j value has 2-adic valuation 4 >= 0
vD
# valuation of discriminant is 14
vc4
# valuation of c_4 is 6, so I need to check Delta' mod 4
Dured
# Delta' is 3 mod 4, so monodromy group is C_6

3

In [869]:
rho = E.galois_representation()
rho.image_type(2)
E.galois_representation?

In [871]:
E.is_ordinary(2)

False

In [878]:
########### playing with some examples ###########

In [18]:
for t in m_k_egs:
    print t, m_k_egs[t]

2 [(II*, '176b1'), (I4*, '240d8'), (II, '704k3'), (I8*, '960i8')]
3 [(IV*, '20a1'), (IV, '20a2')]
4 [(III, '256a1'), (III*, '256a2')]
6 [(II, '80b4'), (I0*, '80b3'), (I4*, '320f2'), (II*, '704f1')]
8 [(I3*, '32a1'), (I0*, '32a3'), (III, '32a2'), (I2*, '64a1'), (I5*, '64a3'), (II, '64a4'), (III*, '256b2')]
24 [(I1*, '24a1'), (III*, '24a3'), (II*, '24a5'), (III, '24a4'), (I0*, '48a1'), (I2*, '48a3'), (I3*, '48a5'), (II, '48a4'), (I7*, '192d5'), (I6*, '192d4'), (I4*, '192c2')]


In [32]:
for (ktype, label) in m_k_egs[2]:
    print("Label: {}, Kodaira type: {}".format(label, ktype))
    E = EllipticCurve(label)
    for d in [-1, -2, 3, 6, -3, -6, 2]:
        K.<a> = NumberField(x^2-d)
        EK = E.base_extend(K)
        pp = K.prime_above(2)
        if EK.has_good_reduction(pp):
            print("Good reduction over Q2 adjoin sqrt of {}".format(d)) 
    print

Label: 176b1, Kodaira type: II*
Good reduction over Q2 adjoin sqrt of -1
Good reduction over Q2 adjoin sqrt of 3

Label: 240d8, Kodaira type: I4*
Good reduction over Q2 adjoin sqrt of -1
Good reduction over Q2 adjoin sqrt of 3

Label: 704k3, Kodaira type: II
Good reduction over Q2 adjoin sqrt of -2
Good reduction over Q2 adjoin sqrt of 6

Label: 960i8, Kodaira type: I8*
Good reduction over Q2 adjoin sqrt of -2
Good reduction over Q2 adjoin sqrt of 6



In [982]:
############ try Q8 example ##############

In [1016]:
f(x) = x^8 - 72*x^6 + 180*x^4 - 144*x^2 + 36; f
F.<a> = NumberField(f); F
G = F.galois_group(); G
pp = F.prime_above(2); 
G2 = G.decomposition_group(pp)
H = QuaternionGroup() 
print G2.is_isomorphic(H)

True


In [1017]:
E = EllipticCurve('32a1');
EF = E.base_extend(F)
EF.local_data(pp)

Local data at Fractional ideal (2, -1/48*a^7 + 17/12*a^5 + 17/8*a^3 - 7/2*a + 1):
Reduction type: bad additive
Local minimal model: Elliptic Curve defined by y^2 + (-3/8*a^7+5/24*a^6+641/24*a^5-59/4*a^4-187/4*a^3+79/4*a^2+79/4*a-7/2)*x*y + (-1/3*a^7-1/6*a^6+191/8*a^5+287/24*a^4-51*a^3-27*a^2+103/4*a+57/4)*y = x^3 + (17/48*a^7+1/24*a^6-607/24*a^5-23/8*a^4+391/8*a^3-5/4*a^2-89/4*a+5/4)*x^2 + (-196526980439434211/24*a^7-69446044139155675/12*a^6+14050842341621224993/24*a^5+9930395360997536039/24*a^4-4714942350467599317/4*a^3-1667294071479135567/2*a^2+2339614203566200403/4*a+1656301670482515941/4)*x + (-124031927880385215/16*a^7-22000355722555773/4*a^6+4433914384012441889/8*a^5+1572946639350029673/4*a^4-8929684704131705171/8*a^3-1583978407796265877/2*a^2+2216409200867391457/4*a+786351063793724633/2) over Number Field in a with defining polynomial x^8 - 72*x^6 + 180*x^4 - 144*x^2 + 36
Minimal discriminant valuation: 12
Conductor exponent: 6
Kodaira Symbol: I2*
Tamagawa Number: 4

In [19]:
# the one quaternionic extension i found by googling did not do the trick 🤷
# but i should be able to construct more because Q8 in Ext(C_2, C_4) and Ext(C_2^2, C_2)

In [100]:
# C6 examples
K3.<a> = NumberField(x^3-2)
for (ktype, label) in m_k_egs[6]:
    print("Label: {}, Kodaira type: {}".format(label, ktype))
    E = EllipticCurve(label)
    for d in [-1, -2, 3, 6, -3, -6, 2]:
        K.<b> = K3.extension(x^2-d)
        EK = E.base_extend(K)
        pp = K.prime_above(2)
        if EK.has_good_reduction(pp):
            print("Good reduction over Q_2(3) adjoin sqrt of {}".format(d)) 
    print

Label: 80b4, Kodaira type: II
Good reduction over Q_2(3) adjoin sqrt of -1
Good reduction over Q_2(3) adjoin sqrt of 3

Label: 80b3, Kodaira type: I0*
Good reduction over Q_2(3) adjoin sqrt of -1
Good reduction over Q_2(3) adjoin sqrt of 3

Label: 320f2, Kodaira type: I4*
Good reduction over Q_2(3) adjoin sqrt of -2
Good reduction over Q_2(3) adjoin sqrt of 6

Label: 704f1, Kodaira type: II*
Good reduction over Q_2(3) adjoin sqrt of -2
Good reduction over Q_2(3) adjoin sqrt of 6



In [132]:
# next step: properties of representation on 320f2 on intermediate extensions
# can compute things over Q2
E = EllipticCurve('320f2')
rho = E.galois_representation()
print(rho.is_irreducible(2))
print(rho.image_type(2))
print(rho.image_classes(2))
print(rho.is_unipotent(2, 3))

False
The image is cyclic of order 2 as there is exactly one rational 2-torsion point.
[1.000, 0.0000]
True


In [182]:
K3.<a> = NumberField(x^3-2)
K.<b> = K3.extension(x^2+2)
EK = E.base_extend(K)
rhoK = EK.galois_representation()
pp = K.prime_above(2)

In [172]:
E.torsion_points()

[(0 : 1 : 0), (1 : 0 : 1)]

In [173]:
print(EK.torsion_points())
print([P.order() for P in EK.torsion_points()])

[(0 : 1 : 0), (-9 : -20*b : 1), (-1 : -4*b : 1), (1 : 0 : 1), (-1 : 4*b : 1), (-9 : 20*b : 1)]
[1, 6, 3, 2, 3, 6]
